In [3]:
import warnings
warnings.filterwarnings('ignore')

import koreanize_matplotlib

In [5]:
import pandas as pd

drop_cols = ['T2', 'T_x_rain', 'RH2', '습도(%)', '풍속(m/s)', 'is_rain', 'wind_level', '강수량(mm)', '기온(°C)', 'ess_hours']

# 경로는 네 환경에 맞춰 수정
train = pd.read_csv("Data/train_0821withcluster.csv").drop(drop_cols,axis = 1)
test  = pd.read_csv("Data/test_0821withcluster.csv").drop(drop_cols,axis = 1)

train = train.drop(columns=[c for c in train.columns if c.startswith("btype_")], errors="ignore")
test  = test.drop(columns=[c for c in test.columns if c.startswith("btype_")], errors="ignore")

train

,num_date_time,건물번호,일시,일조(hr),일사(MJ/m2),전력소비량(kWh),건물유형,연면적(m2),냉방면적(m2),태양광용량(kW),...,hour_sin,hour_cos,month_sin,month_cos,T_x_RH,cooling_load_index,time_usage_level,day_usage_level,cluster_id,기온_MA3
0,1_20240601 00,1,2024-06-01 00:00:00,0.0,0.00,5794.80,호텔,82912.71,77586.0,0.0,...,0.000000,1.000000,1.224647e-16,-1.0,1500.6,60.125023,0,1,2,18.300000
1,1_20240601 01,1,2024-06-01 01:00:00,0.0,0.00,5591.85,호텔,82912.71,77586.0,0.0,...,0.258819,0.965926,1.224647e-16,-1.0,1500.6,60.125023,0,1,2,18.300000
2,1_20240601 02,1,2024-06-01 02:00:00,0.0,0.00,5338.17,호텔,82912.71,77586.0,0.0,...,0.500000,0.866025,1.224647e-16,-1.0,1448.0,59.753772,0,1,2,18.233333
3,1_20240601 03,1,2024-06-01 03:00:00,0.0,0.00,4554.42,호텔,82912.71,77586.0,0.0,...,0.707107,0.707107,1.224647e-16,-1.0,1458.0,59.636802,-1,1,2,18.133333
4,1_20240601 04,1,2024-06-01 04:00:00,0.0,0.00,3602.25,호텔,82912.71,77586.0,0.0,...,0.866025,0.500000,1.224647e-16,-1.0,1441.8,59.335133,-1,1,2,17.966667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203995,100_20240824 19,100,2024-08-24 19:00:00,0.4,0.18,3276.00,호텔,162070.24,152943.0,0.0,...,-0.965926,0.258819,-8.660254e-01,-0.5,2211.6,81.722294,0,1,0,30.233333
203996,100_20240824 20,100,2024-08-24 20:00:00,0.0,0.00,3197.52,호텔,162070.24,152943.0,0.0,...,-0.866025,0.500000,-8.660254e-01,-0.5,2116.4,80.220593,0,1,0,29.366667
203997,100_20240824 21,100,2024-08-24 21:00:00,0.0,0.00,3006.60,호텔,162070.24,152943.0,0.0,...,-0.707107,0.707107,-8.660254e-01,-0.5,2094.2,79.483875,0,1,0,28.666667
203998,100_20240824 22,100,2024-08-24 22:00:00,0.0,0.00,2649.72,호텔,162070.24,152943.0,0.0,...,-0.500000,0.866025,-8.660254e-01,-0.5,2128.0,79.000442,1,1,0,28.300000


In [4]:
train['cluster_id'].value_counts()

cluster_id
0     65280
10    46920
11    36720
4     16320
2      8160
8      8160
1      8160
6      4080
5      4080
9      2040
7      2040
3      2040
Name: count, dtype: int64

In [7]:
train.to_csv('Data/train_0821_ensemble.csv',index = False)
test.to_csv('Data/test_0821_ensemble.csv',index = False)

## 앙상블 단계별 비교

In [10]:
import numpy as np
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor, Pool
import xgboost as xgb

# ======================
# 0) 경로 & 상수
# ======================
TRAIN_PATH = "Data/train_0821_ensemble.csv"
TEST_PATH  = "Data/test_0821_ensemble.csv"
SUB_PATH   = "Data/sample_submission.csv"

KEYS   = ["num_date_time", "건물번호"]
TARGET = "전력소비량(kWh)"
BASE_DROP_COMMON = ["num_date_time", "일시"]  # 공통 시간 문자열
BASE_DROP_TRAIN  = ["건물유형", TARGET]       # 모델1·3 (건물유형 split)에서 drop
BASE_DROP_TEST   = ["건물유형"]

train = pd.read_csv(TRAIN_PATH)
test  = pd.read_csv(TEST_PATH)
sub   = pd.read_csv(SUB_PATH)


In [11]:
def smape(y_true, y_pred):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    denom = (np.abs(y_true) + np.abs(y_pred))
    denom = np.where(denom == 0, 1, denom)
    return np.mean(2.0 * np.abs(y_pred - y_true) / denom) * 100

def coerce_numeric_objects(df: pd.DataFrame) -> pd.DataFrame:
    """object/category 컬럼을 안전하게 숫자로 변환 (숫자/기호 혼재 대응)"""
    out = df.copy()
    for c in out.columns:
        if str(out[c].dtype) in ["object", "category"]:
            out[c] = pd.to_numeric(
                out[c].astype(str).str.replace(r"[^\d\.\-eE]", "", regex=True),
                errors="coerce"
            )
    return out

def reorder_like(X_to_fix, X_ref):
    """X_to_fix를 X_ref의 컬럼 순서/구성에 맞춘다. 누락은 NaN으로 채움."""
    return X_to_fix.reindex(columns=X_ref.columns, fill_value=np.nan)

In [25]:
# 결과 수집 
# ======================
# 모델 1: CatBoost (건물유형별)
val_oofs_1, test_preds_1 = [], []
# 모델 2: XGBoost (cluster_id별)
val_oofs_2, test_preds_2 = [], []
# 모델 3: XGBoost (건물유형별)
val_oofs_3, test_preds_3 = [], []

val_scores_1 = {}
val_scores_2 = {}
val_scores_3 = {}

In [26]:
# ======================
# 4) 모델 1 — CatBoost (건물유형별)
# ======================
building_types = train["건물유형"].dropna().unique()
params_cb = dict(
    loss_function="RMSE", 
    eval_metric="SMAPE",
    learning_rate=0.05,
    depth=8,
    l2_leaf_reg=3.0,
    random_seed=42,
    iterations=2500,
    od_type="Iter",
    od_wait=200,
    verbose=False,
    allow_writing_files=False
)

for btype in building_types:
    tr_sub = train[train["건물유형"] == btype].copy()
    te_sub = test [test ["건물유형"] == btype].copy()
    if len(tr_sub) == 0:
        continue

    drop_cols_tr = BASE_DROP_COMMON + BASE_DROP_TRAIN
    X_all = tr_sub.drop(columns=[c for c in drop_cols_tr if c in tr_sub.columns], errors="ignore")
    y_all = tr_sub[TARGET].astype(float)

    # 학습/검증 분리
    X_tr, X_val, y_tr, y_val = train_test_split(X_all, y_all, test_size=0.2, random_state=42)

    # cat_features: object/category를 그대로 사용
    cat_cols = [c for c in X_tr.columns if str(X_tr[c].dtype) in ["object", "category"]]
    pool_tr  = Pool(X_tr, y_tr, cat_features=cat_cols)
    pool_val = Pool(X_val, y_val, cat_features=cat_cols)

    m1 = CatBoostRegressor(**params_cb)
    m1.fit(pool_tr, eval_set=pool_val, use_best_model=True)

    # OOF 수집
    val_pred = m1.predict(pool_val)
    score = smape(y_val.values, val_pred)
    val_scores_1[btype] = score   # 건물유형 단위 성능 기록
    print(f"[{btype}] Validation SMAPE: {score:.4f}")
    
    val_keys = tr_sub.loc[X_val.index, KEYS].reset_index(drop=True)
    oof_df = val_keys.copy()
    oof_df["y_true"] = y_val.reset_index(drop=True).values
    oof_df["pred_1"] = val_pred
    val_oofs_1.append(oof_df)

    # TEST 예측
    X_test = te_sub.drop(columns=[c for c in BASE_DROP_COMMON + BASE_DROP_TEST if c in te_sub.columns], errors="ignore")
    X_test = reorder_like(X_test, X_all)
    pool_te = Pool(X_test, cat_features=cat_cols)
    te_pred = m1.predict(pool_te)
    test_preds_1.append(te_sub[KEYS].assign(pred_1=te_pred))


[호텔] Validation SMAPE: 5.8770
[상용] Validation SMAPE: 2.3840
[병원] Validation SMAPE: 2.8614
[학교] Validation SMAPE: 2.8401
[건물기타] Validation SMAPE: 5.0446
[아파트] Validation SMAPE: 9.5307
[연구소] Validation SMAPE: 4.7642
[백화점] Validation SMAPE: 5.7838
[IDC(전화국)] Validation SMAPE: 1.0828
[공공] Validation SMAPE: 5.1780


In [30]:
# ======================
# 모델 2 — XGBoost (cluster_id별)
# ======================
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split

# object/category → float 변환 유틸 (있다면 기존 coerce_numeric_objects 써도 됨)
def to_numeric_df(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    for c in out.columns:
        if str(out[c].dtype) in ["object", "category"]:
            out[c] = pd.to_numeric(
                out[c].astype(str).str.replace(r"[^\d\.\-eE]", "", regex=True),
                errors="coerce"
            )
    return out

params_xgb = dict(
    n_estimators=700,
    learning_rate=0.05,
    max_depth=10,
    subsample=0.8,
    colsample_bytree=0.8,
    tree_method="hist",  # GPU면 "gpu_hist"
    random_state=42,
    n_jobs=-1,
    eval_metric="mae"
)

clusters = train["cluster_id"].dropna().unique() if "cluster_id" in train.columns else []
for cid in clusters:
    tr_sub = train[train["cluster_id"] == cid].copy()
    te_sub = test [test ["cluster_id"] == cid].copy()
    if len(tr_sub) == 0:
        continue

    # 1) Feature/Target
    drop_cols_tr = BASE_DROP_COMMON + ["cluster_id", '건물번호', TARGET]
    X_all = tr_sub.drop(columns=[c for c in drop_cols_tr if c in tr_sub.columns], errors="ignore")
    y_all = tr_sub[TARGET].astype(float)

    # XGBoost는 숫자만 허용 → 변환 + 결측 보정
    X_all = to_numeric_df(X_all).fillna(0.0)

    # 2) Split
    X_tr, X_val, y_tr, y_val = train_test_split(X_all, y_all, test_size=0.2, random_state=38)

    # 3) 모델 학습 (조기 종료)
    m2 = xgb.XGBRegressor(**params_xgb)
    m2.fit(
        X_tr, y_tr,
        eval_set=[(X_val, y_val)],
        verbose=False
    )

    # 4) OOF 수집
    val_pred = m2.predict(X_val)
    score = smape(y_val.values, val_pred)
    val_scores_2[cid] = score   # 건물유형 단위 성능 기록
    print(f"[{cid}] Validation SMAPE: {score:.4f}")

    val_keys = tr_sub.loc[X_val.index, KEYS].reset_index(drop=True)
    oof_df = val_keys.copy()
    oof_df["y_true"] = y_val.reset_index(drop=True).values
    oof_df["pred_2"] = val_pred
    val_oofs_2.append(oof_df)

    # 5) TEST 예측
    X_test = te_sub.drop(columns=[c for c in BASE_DROP_COMMON + ["cluster_id"] if c in te_sub.columns], errors="ignore")
    X_test = to_numeric_df(X_test).fillna(0.0)
    # 학습 컬럼 정합성(순서/누락) 맞추기
    X_test = X_test.reindex(columns=X_tr.columns, fill_value=0.0)

    te_pred = m2.predict(X_test)
    test_preds_2.append(te_sub[KEYS].assign(pred_2=te_pred))


[2] Validation SMAPE: 2.6798
[0] Validation SMAPE: 4.7166
[9] Validation SMAPE: 2.2084
[10] Validation SMAPE: 5.0498
[4] Validation SMAPE: 2.6558
[11] Validation SMAPE: 5.3440
[8] Validation SMAPE: 2.5135
[1] Validation SMAPE: 0.8612
[7] Validation SMAPE: 2.3247
[6] Validation SMAPE: 0.6728
[5] Validation SMAPE: 4.2153
[3] Validation SMAPE: 1.0102


In [28]:
# ======================
# 모델 3 — XGBoost (건물유형별)
# ======================


for btype in building_types:
    tr_sub = train[train["건물유형"] == btype].copy()
    te_sub = test [test ["건물유형"] == btype].copy()
    if len(tr_sub) == 0:
        continue

    drop_cols_tr = BASE_DROP_COMMON + BASE_DROP_TRAIN
    X_all = tr_sub.drop(columns=[c for c in drop_cols_tr if c in tr_sub.columns], errors="ignore")
    y_all = tr_sub[TARGET].astype(float)

    # XGBoost는 float만 → object 수치 변환
    X_all = coerce_numeric_objects(X_all).fillna(0.0)

    X_tr, X_val, y_tr, y_val = train_test_split(X_all, y_all, test_size=0.2, random_state=42)

    m3 = xgb.XGBRegressor(**params_xgb)
    m3.fit(X_tr, y_tr, eval_set=[(X_val, y_val)], verbose=False)

    val_pred = m3.predict(X_val)

    score = smape(y_val.values, val_pred)
    val_scores_3[btype] = score   # 건물유형 단위 성능 기록
    print(f"[{btype}] Validation SMAPE: {score:.4f}")

    val_keys = tr_sub.loc[X_val.index, KEYS].reset_index(drop=True)
    oof_df = val_keys.copy()
    oof_df["y_true"] = y_val.reset_index(drop=True).values
    oof_df["pred_3"] = val_pred
    val_oofs_3.append(oof_df)

    # TEST
    X_test = te_sub.drop(columns=[c for c in BASE_DROP_COMMON + BASE_DROP_TEST if c in te_sub.columns], errors="ignore")
    X_test = coerce_numeric_objects(X_test).fillna(0.0)
    X_test = reorder_like(X_test, X_all).fillna(0.0)
    te_pred = m3.predict(X_test)
    test_preds_3.append(te_sub[KEYS].assign(pred_3=te_pred))

[호텔] Validation SMAPE: 5.2199
[상용] Validation SMAPE: 2.1598
[병원] Validation SMAPE: 2.7611
[학교] Validation SMAPE: 2.8961
[건물기타] Validation SMAPE: 4.7588
[아파트] Validation SMAPE: 5.0580
[연구소] Validation SMAPE: 3.9211
[백화점] Validation SMAPE: 4.8733
[IDC(전화국)] Validation SMAPE: 0.8825
[공공] Validation SMAPE: 4.5050


In [31]:
# 모델 1 결과 확인
final_preds_1 = pd.concat(test_preds_1, axis=0).sort_values(['num_date_time', '건물번호'])
print("=== Model 1 (CatBoost-건물유형) ===")
print("Validation SMAPE by type:", {k: round(v, 4) for k, v in val_scores_1.items()})
print("평균 SMAPE:", round(np.mean(list(val_scores_1.values())), 4))

# 모델 2 결과 확인
final_preds_2 = pd.concat(test_preds_2, axis=0).sort_values(['num_date_time', '건물번호'])
print("\n=== Model 2 (CatBoost-cluster_id) ===")
print("Validation SMAPE by cluster:", {k: round(v, 4) for k, v in val_scores_2.items()})
print("평균 SMAPE:", round(np.mean(list(val_scores_2.values())), 4))

# 모델 3 결과 확인
final_preds_3 = pd.concat(test_preds_3, axis=0).sort_values(['num_date_time', '건물번호'])
print("\n=== Model 3 (XGBoost-건물유형) ===")
print("Validation SMAPE by type:", {k: round(v, 4) for k, v in val_scores_3.items()})
print("평균 SMAPE:", round(np.mean(list(val_scores_3.values())), 4))


=== Model 1 (CatBoost-건물유형) ===
Validation SMAPE by type: {'호텔': np.float64(5.877), '상용': np.float64(2.384), '병원': np.float64(2.8614), '학교': np.float64(2.8401), '건물기타': np.float64(5.0446), '아파트': np.float64(9.5307), '연구소': np.float64(4.7642), '백화점': np.float64(5.7838), 'IDC(전화국)': np.float64(1.0828), '공공': np.float64(5.178)}
평균 SMAPE: 4.5346

=== Model 2 (CatBoost-cluster_id) ===
Validation SMAPE by cluster: {'공공': np.float64(0.9949), np.int64(2): np.float64(2.6798), np.int64(0): np.float64(4.7166), np.int64(9): np.float64(2.2084), np.int64(10): np.float64(5.0498), np.int64(4): np.float64(2.6558), np.int64(11): np.float64(5.344), np.int64(8): np.float64(2.5135), np.int64(1): np.float64(0.8612), np.int64(7): np.float64(2.3247), np.int64(6): np.float64(0.6728), np.int64(5): np.float64(4.2153), np.int64(3): np.float64(1.0102)}
평균 SMAPE: 2.7113

=== Model 3 (XGBoost-건물유형) ===
Validation SMAPE by type: {'호텔': np.float64(5.2199), '상용': np.float64(2.1598), '병원': np.float64(2.7611), '학교': np.f

In [32]:
# ======================
# 7) OOF 결합 + 가중치 최적화 (SMAPE 최소)
# ======================
def concat_if_any(lst):
    return pd.concat(lst, ignore_index=True) if len(lst) else pd.DataFrame(columns=KEYS+["y_true"])

oof1 = concat_if_any(val_oofs_1)
oof2 = concat_if_any(val_oofs_2)
oof3 = concat_if_any(val_oofs_3)

# inner merge: 동일 샘플만 비교
val_join = oof1.merge(oof2, on=KEYS+["y_true"], how="inner") if len(oof2) else oof1.copy()
val_join = val_join.merge(oof3, on=KEYS+["y_true"], how="inner") if len(oof3) else val_join

# pred 컬럼 존재 확인
has1 = "pred_1" in val_join
has2 = "pred_2" in val_join
has3 = "pred_3" in val_join

assert has1 or has2 or has3, "OOF 예측이 없습니다. (최소 한 모델은 필요)"

y_true = val_join["y_true"].values
stack = []
names = []
if has1:
    stack.append(val_join["pred_1"].values); names.append("1")
if has2:
    stack.append(val_join["pred_2"].values); names.append("2")
if has3:
    stack.append(val_join["pred_3"].values); names.append("3")
P = np.vstack(stack)  # shape (M, N), M=사용 모델 수

# 단순 그리드 탐색 (Σw=1, w≥0)
grid = np.linspace(0, 1, 51)  # 0.02 간격
best = {"w": None, "smape": 1e9}
if P.shape[0] == 1:
    # 모델 하나만 있으면 가중치=1
    best = {"w": (1.0,), "smape": smape(y_true, P[0])}
else:
    # M=2 또는 3 케이스 모두 대응
    if P.shape[0] == 2:
        for w1 in grid:
            w2 = 1.0 - w1
            if w2 < 0: continue
            w = np.array([w1, w2])
            y_hat = np.dot(w, P)
            s = smape(y_true, y_hat)
            if s < best["smape"]:
                best = {"w": tuple(w), "smape": s}
    else:  # M == 3
        for w1 in grid:
            for w2 in grid:
                w3 = 1.0 - w1 - w2
                if w3 < 0: continue
                w = np.array([w1, w2, w3])
                y_hat = np.dot(w, P)
                s = smape(y_true, y_hat)
                if s < best["smape"]:
                    best = {"w": tuple(w), "smape": s}

print(f"[OOF 기반 최적 가중치] 사용모델={names} | w={best['w']} | OOF SMAPE={best['smape']:.4f}")


[OOF 기반 최적 가중치] 사용모델=['1', '2', '3'] | w=(np.float64(0.08), np.float64(0.28), np.float64(0.64)) | OOF SMAPE=3.6974


In [34]:
# ======================
# TEST 결합 + 가중 앙상블 (FIXED)
# ======================

def join_test_preds(test_list, cname: str):
    """test_list: List[pd.DataFrame]  (각 DF는 KEYS + [cname])
       cname: 'pred_1' / 'pred_2' / 'pred_3'
       반환: KEYS 기준으로 중복 평균 집계된 단일 DF
    """
    if not test_list or len(test_list) == 0:
        return None
    merged = pd.concat(test_list, ignore_index=True)            # ✅ 리스트를 concat
    merged = merged.groupby(KEYS, as_index=False)[cname].mean() # 중복 키 평균
    return merged

t1 = join_test_preds(test_preds_1, "pred_1") if len(test_preds_1) else None
t2 = join_test_preds(test_preds_2, "pred_2") if len(test_preds_2) else None
t3 = join_test_preds(test_preds_3, "pred_3") if len(test_preds_3) else None

# 최종 test merge
test_join = None
for t in (t1, t2, t3):
    if t is None:
        continue
    test_join = t if test_join is None else test_join.merge(t, on=KEYS, how="outer")

# 없는 pred 컬럼은 0으로 채움 (사용 모델 수에 맞춰)
for cname in ["pred_1", "pred_2", "pred_3"]:
    if cname not in test_join.columns:
        test_join[cname] = 0.0

# 사용 모델 여부(안전하게 다시 계산)
has1 = ("pred_1" in test_join.columns) and (t1 is not None)
has2 = ("pred_2" in test_join.columns) and (t2 is not None)
has3 = ("pred_3" in test_join.columns) and (t3 is not None)

# 가중치 배열 정리 (numpy 스칼라 → float)
w = tuple(float(x) for x in best["w"])

pred_stack = []
if has1: pred_stack.append(test_join["pred_1"].values)
if has2: pred_stack.append(test_join["pred_2"].values)
if has3: pred_stack.append(test_join["pred_3"].values)
pred_stack = np.vstack(pred_stack)   # shape (M, N)

# w도 M 길이여야 함 (모델 수 M에 맞게 슬라이스)
w = np.array(w[:pred_stack.shape[0]])

# 가중합
ens = np.dot(w, pred_stack)
test_join["pred_ens"] = np.clip(ens, a_min=0, a_max=None)


In [38]:
test_join[['num_date_time', 'pred_ens']].sort_values('num_date_time')

,num_date_time,pred_ens
0,100_20240825 00,2857.110068
1,100_20240825 01,2829.224095
2,100_20240825 02,2668.614004
3,100_20240825 03,2192.869884
4,100_20240825 04,2462.317576
...,...,...
16795,9_20240831 19,3195.688172
16796,9_20240831 20,2941.183310
16797,9_20240831 21,2683.416860
16798,9_20240831 22,2518.186966


In [49]:
def apply_min_clip_by_weekday_hour(train: pd.DataFrame,
                                   test: pd.DataFrame,
                                   preds_df: pd.DataFrame,
                                   pred_col: str = "pred_ens",
                                   target_col: str = "전력소비량(kWh)") -> pd.DataFrame:
    """6-8월 train에서 (건물번호, 요일, 시)별 타깃의 '최저값'을 구해
       예측이 그 값보다 작으면 교체(상향)한다.
    """
    # 1) train에서 요일/시/월 파생
    tr = train.copy()
    tr["dt"] = pd.to_datetime(tr["일시"])
    tr["month"] = tr["dt"].dt.month
    tr["weekday"] = tr["dt"].dt.dayofweek
    tr["hour"] = tr["dt"].dt.hour

    # 2) 6~8월만 필터
    tr_summer = tr[(tr["month"].between(6, 8))].copy()

    # 3) (건물번호, weekday, hour)별 최저 타깃
    grp_min = (tr_summer
               .groupby(["건물번호", "weekday", "hour"], as_index=False)[target_col]
               .min()
               .rename(columns={target_col: "min_hist"}))

    # 4) test에도 요일/시 파생
    te = test[["num_date_time", "건물번호", "일시"]].copy()
    te["dt"] = pd.to_datetime(te["일시"])
    te["weekday"] = te["dt"].dt.dayofweek
    te["hour"] = te["dt"].dt.hour
    te = te[["num_date_time", "건물번호", "weekday", "hour"]]

    # 5) preds_df와 key merge
    base = preds_df.merge(te, on=["num_date_time", "건물번호"], how="left")

    # 6) (건물번호, weekday, hour)로 min_hist merge
    base = base.merge(grp_min, on=["건물번호", "weekday", "hour"], how="left")

    # 7) 하한 클리핑
    before = base[pred_col].copy()
    base[pred_col] = np.where(
        (~base["min_hist"].isna()) & (base[pred_col] < base["min_hist"]),
        base["min_hist"],
        base[pred_col]
    )

    # 차이 계산
    diff = (base[pred_col] - before).clip(lower=0)  # 교체된 만큼의 증가량
    changed = int((diff > 0).sum())
    total_increase = diff.sum()

    print(f"[Clip] {changed} rows were raised. 누적 증가량={total_increase:.2f}")

    return base[preds_df.columns]

In [50]:
clipped_preds = apply_min_clip_by_weekday_hour(train, test, test_join[["num_date_time",'건물번호',"pred_ens"]], pred_col="pred_ens")
clipped_preds

[Clip] 149 rows were raised. 누적 증가량=63628.75


,num_date_time,건물번호,pred_ens
0,100_20240825 00,100,2857.110068
1,100_20240825 01,100,2829.224095
2,100_20240825 02,100,2668.614004
3,100_20240825 03,100,2192.869884
4,100_20240825 04,100,2462.317576
...,...,...,...
16795,9_20240831 19,9,3195.688172
16796,9_20240831 20,9,2941.183310
16797,9_20240831 21,9,2683.416860
16798,9_20240831 22,9,2518.186966


In [52]:
submit = pd.read_csv('Data/sample_submission.csv')

sub_out = submit.merge(
    clipped_preds[['num_date_time', 'pred_ens']],
    on='num_date_time',
    how="left"
).copy()

# 컬럼 이름 변경
sub_out = sub_out[['num_date_time', 'pred_ens']].rename(columns={'pred_ens': 'answer'})

# 최종 제출 파일 저장
sub_out.to_csv("Submit/submit0821_가중앙상블+클리핑3.6974.csv", index=False)
sub_out

,num_date_time,answer
0,1_20240825 00,4476.336000
1,1_20240825 01,4177.128251
2,1_20240825 02,3848.795925
3,1_20240825 03,3283.351214
4,1_20240825 04,3037.355728
...,...,...
16795,100_20240831 19,2665.438578
16796,100_20240831 20,2696.499738
16797,100_20240831 21,2442.366293
16798,100_20240831 22,2549.769203
